In [1]:
%classpath add jar konduit.jar

In [2]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.pipeline.api.data.Data;
import ai.konduit.serving.pipeline.api.data.Image;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;
import ai.konduit.serving.pipeline.impl.format.JavaImageFactory;
import ai.konduit.serving.pipeline.registry.ImageFactoryRegistry;
import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

public class OCREndPoint implements Endpoint {

    private PipelineExecutor pipelineExecutor;

    public OCREndPoint(PipelineExecutor pipelineExecutor) { this.pipelineExecutor = pipelineExecutor; ImageFactoryRegistry.addFactory(new JavaImageFactory()); }

    public HttpMethod type() { return HttpMethod.POST; }

    public String path() { return "/infer"; }

    public List<String> consumes() { return Arrays.asList("application/octet-stream","multipart/form-data"); }

    public List<String> produces() { return Arrays.asList("application/json"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> {
            handler.vertx().executeBlocking(taskHandler -> {
                Data image = Data.empty();
                
                try {
                    image.put("image",Image.create(ImageIO.read(new File(handler.fileUploads().iterator().next().uploadedFileName()))));
                } catch (IOException e) {
                    e.printStackTrace();
                }

                Data exec = pipelineExecutor.exec(image);
                
                handler.response().end(exec.toJson());
                taskHandler.complete();
            },resultHandler -> {
                if(resultHandler.failed()) {
                    if(resultHandler.cause() != null)
                        if(handler.vertx().exceptionHandler() != null)
                            handler.vertx().exceptionHandler().handle(resultHandler.cause());
                        else {
                            resultHandler.cause().printStackTrace();
                        }
                    else {
                        System.err.println("Failed to process classification endpoint async task. Unknown cause.");
                    }
                }
            });

        };
    }
}

ai.konduit.OCREndPoint

In [3]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.endpoint.HttpEndpoints;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import java.util.Arrays;
import java.util.List;

public class OCREndPoints implements HttpEndpoints {

    @Override
    public List<Endpoint> endpoints(Pipeline pipeline, PipelineExecutor pipelineExecutor) {
        return Arrays.asList(new OCREndPoint(pipelineExecutor));
    }
}

ai.konduit.OCREndPoints

In [4]:
import java.net.URLClassLoader;
import java.net.URL;

URL[] urls = ((URLClassLoader) Class.forName("ai.konduit.serving.vertx.config.InferenceConfiguration").getClassLoader()).getURLs();

for(URL url : urls) {
    System.out.println(new java.io.File(url.toURI()));
}

C:\Users\shams\AppData\Local\Temp\beaker220133109328875192\outDir
E:\Demos\nec\konduit.jar


null

In [ ]:
%%bash
alias konduit=bin/konduit

In [ ]:
%%bash
java -cp /tmp/beaker6984068703259125692/outDir/:konduit.jar ai.konduit.serving.cli.launcher.KonduitServingLauncher serve -id server -c config.json -rwm -b